#### Elliptic Curve


An elliptic curve is a smooth, projective, algebraic curve of genus one with a specified point $ \mathcal{O} $ (called the point at infinity), defined over a field $ \mathbb{K} $. The general equation of an elliptic curve over a field $ \mathbb{K} $ is given by:

$$
y^2 = x^3 + ax + b
$$

where $ a, b \in \mathbb{K} $ and the curve must satisfy the non-singularity condition, which means its discriminant $ \Delta $ must be non-zero:

$$
\Delta = 4a^3 + 27b^2 \neq 0
$$

This equation defines the set of points $ (x, y) $ on the elliptic curve, along with the point at infinity $ \mathcal{O} $, which serves as the identity element in the elliptic curve's group structure.

#### Elliptic Curve over Finite Fields

For cryptographic purposes, elliptic curves are typically defined over finite fields $ \mathbb{F}_p $, where $ p $ is a prime number. An elliptic curve over a finite field $ \mathbb{F}_p $ is represented as:

$$
E(\mathbb{F}_p) = \{(x, y) \in \mathbb{F}_p^2 \mid y^2 = x^3 + ax + b\} \cup \{\mathcal{O}\}
$$

The set of points $ E(\mathbb{F}_p) $ is finite, and the number of points on such a curve is approximated by $ p + 1 $, with more precise bounds provided by the Hasse theorem.

## Class Overview: `EllipticCurve`

The `EllipticCurve` class includes methods for performing key operations on elliptic curves over finite fields:

1. **Point Verification**:
   - The method `is_point_on_curve(x, y)` checks whether a point $ (x, y) $ lies on the elliptic curve.

2. **Point Addition**:
   - The method `point_addition(P, Q)` implements the elliptic curve addition law, allowing for the addition of two points $ P $ and $ Q $ on the curve. This method handles:
     - The identity point (point at infinity).
     - The addition of distinct points.
     - The doubling of a point.

3. **Point Multiplication**:
   - The method `point_multiply(n, P)` uses the Double and Add algorithm to compute the scalar multiplication of a point $ P $ by an integer $ n $.

With these methods, you can perform essential operations on elliptic curves, which are fundamental in cryptography and number theory.


In [1]:
class EllipticCurve:
    def __init__(self, p, a, b):
        self.p = p
        self.a = a
        self.b = b

    def is_point_on_curve(self, x, y):
        return (y**2 - x**3 - self.a*x - self.b) % self.p == 0

    def point_addition(self, P, Q):
        if P is None:
            return Q
        if Q is None:
            return P
        if P[0] == Q[0] and P[1] != Q[1]:
            return None
        if P != Q:
            lam = (Q[1] - P[1]) * pow(Q[0] - P[0], -1, self.p) % self.p
        else:
            lam = (3 * P[0]**2 + self.a) * pow(2 * P[1], -1, self.p) % self.p
        x3 = (lam**2 - P[0] - Q[0]) % self.p
        y3 = (lam * (P[0] - x3) - P[1]) % self.p
        return (x3, y3)

    def point_multiply(self, n, P):             # Double and Add Algorithm
        Q = P                                   # Set Q = P and R = O.
        R = None
        while n > 0:
            if n % 2 == 1:                      # If n ≡ 1 (mod 2), set R = R + Q.
                R = self.point_addition(R, Q)
            Q = self.point_addition(Q, Q)
            n //= 2                             # Set Q = 2Q and n = n/2.
        return R

### Addition of Point of an Elliptic Curve over $ F_P $

# Addition of Points on an Elliptic Curve over $ \mathbb{F}_p $

Let $ E $ be an elliptic curve. Then the addition law on $ E $ has the following properties:

1. **Identity**: $ P + \mathcal{O} = \mathcal{O} + P = P $ for all $ P \in E $.
2. **Inverse**: $ P + (-P) = \mathcal{O} $ for all $ P \in E $.
3. **Associative**: $ (P + Q) + R = P + (Q + R) $ for all $ P, Q, R \in E $.
4. **Commutative**: $ P + Q = Q + P $ for all $ P, Q \in E $.

In other words, the addition law makes the points of $ E $ into an abelian group.

## The Elliptic Curve Addition Algorithm

The algorithm describes how to add two points on an elliptic curve $ E $ given by the equation $ E: y^2 = x^3 + Ax + B $.

1. **Identity Points**:
   - If $ P_1 = \mathcal{O} $ (the point at infinity), then $ P_1 + P_2 = P_2 $.
   - If $ P_2 = \mathcal{O} $, then $ P_1 + P_2 = P_1 $.

2. **Opposite Points**:
   - For points $ P_1 = (x_1, y_1) $ and $ P_2 = (x_2, y_2) $:
     - If $ x_1 = x_2 $ and $ y_1 = -y_2 $, then $ P_1 + P_2 = \mathcal{O} $.

3. **Otherwise**:
   - For two distinct points $ P = (x_1, y_1) $ and $ Q = (x_2, y_2) $, the sum $ R = P + Q = (x_3, y_3) $ is defined as follows:
     - Calculate the $ \lambda $, where $ \lambda $ is the slope of the line through $ P $ and $ Q $:
     $$
     \lambda =
     \begin{cases}
     \frac{y_2 - y_1}{x_2 - x_1} & \text{if } P_1 \neq P_2, \\
     \frac{3x_1^2 + A}{2y_1} & \text{if } P_1 = P_2.
     \end{cases}
     $$

     - Compute the coordinates of $ P_3 = P_1 + P_2 = (x_3, y_3) $:
     $$
     x_3 = \lambda^2 - x_1 - x_2,
     $$
     $$
     y_3 = \lambda(x_1 - x_3) - y_1.
     $$


In [6]:
def point_addition(p1, p2, prime, a):
    x1, y1 = p1
    x2, y2 = p2

    if p1 == (None, None):
        return p2
    if p2 == (None, None):
        return p1

    if p1 != p2:
        denominator = (x2 - x1) % prime
        if denominator == 0:
            return (None, None)
        slope = ((y2 - y1) * pow(denominator, -1, prime)) % prime
    else:
        denominator = (2 * y1) % prime
        if denominator == 0:
            return (None, None)
        slope = ((3 * x1 * x1 + a) * pow(denominator, -1, prime)) % prime


    x3 = (slope * slope - x1 - x2) % prime
    y3 = (slope * (x1 - x3) - y1) % prime

    return x3, y3

def generate_points(prime, a, b):
    points = [(None, None)]
    for x in range(prime):
        for y in range(prime):
            if (y*y - (x*x*x + a*x + b)) % prime == 0:
                points.append((x, y))
    return points

def generate_addition_table(prime, a, b):
    points = generate_points(prime, a, b)
    table = [['EC'] + [f'({x},{y})' for x, y in points]]
    for p1 in points:
        row = [f'({p1[0]},{p1[1]})']
        for p2 in points:
            result = point_addition(p1, p2, prime, a)
            row.append(f'({result[0]},{result[1]})')
        table.append(row)
    return table


prime = 17
a = -15
b = 18

#table
addition_table = generate_addition_table(prime, a, b)
for row in addition_table:
    print("\t".join(row))

EC	(None,None)	(0,1)	(0,16)	(1,2)	(1,15)	(2,8)	(2,9)	(3,0)	(5,0)	(6,5)	(6,12)	(7,1)	(7,16)	(8,6)	(8,11)	(9,0)	(10,1)	(10,16)	(12,6)	(12,11)	(14,6)	(14,11)	(16,7)	(16,10)
(None,None)	(None,None)	(0,1)	(0,16)	(1,2)	(1,15)	(2,8)	(2,9)	(3,0)	(5,0)	(6,5)	(6,12)	(7,1)	(7,16)	(8,6)	(8,11)	(9,0)	(10,1)	(10,16)	(12,6)	(12,11)	(14,6)	(14,11)	(16,7)	(16,10)
(0,1)	(0,1)	(1,15)	(None,None)	(0,16)	(8,6)	(6,12)	(14,11)	(16,10)	(10,1)	(2,9)	(12,11)	(10,16)	(8,11)	(7,1)	(1,2)	(12,6)	(7,16)	(5,0)	(6,5)	(9,0)	(2,8)	(16,7)	(3,0)	(14,6)
(0,16)	(0,16)	(None,None)	(1,2)	(8,11)	(0,1)	(14,6)	(6,5)	(16,7)	(10,16)	(12,6)	(2,8)	(8,6)	(10,1)	(1,15)	(7,16)	(12,11)	(5,0)	(7,1)	(9,0)	(6,12)	(16,10)	(2,9)	(14,11)	(3,0)
(1,2)	(1,2)	(0,16)	(8,11)	(7,16)	(None,None)	(16,10)	(12,6)	(14,11)	(7,1)	(9,0)	(14,6)	(1,15)	(5,0)	(0,1)	(10,1)	(6,12)	(10,16)	(8,6)	(12,11)	(2,8)	(3,0)	(6,5)	(2,9)	(16,7)
(1,15)	(1,15)	(8,6)	(0,1)	(None,None)	(7,1)	(12,11)	(16,7)	(14,6)	(7,16)	(14,11)	(9,0)	(5,0)	(1,2)	(10,16)	(0,16)	(6,5)	(8,11)	(10,

#### ECDH


| **Private Parameter Setup** |  |
|-----------------------------|--|
| Choose a large prime $ p $ and a prime-order elliptic curve $ E $ over $ \mathbb{F}_p $. Select a base point $ P $ on $ E(\mathbb{F}_p) $. |  |

| **Private Computation** | **Private Computation** |
|-------------------------|-------------------------|
| **Rick**                | **Morty**               |
| Choose a private integer $ n_R $ | Choose a private integer $ n_M $ |
| Calculate the point $ Q_R = n_R P $ | Calculate the point $ Q_M = n_M P $ |

| **Public Exchange of Values** |  |
|-------------------------------|--|
| Rick sends $ Q_R \longrightarrow $ Morty |  |
| Morty sends $ Q_M \longrightarrow $ Rick |  |

| **Private Computation** | **Private Computation** |
|-------------------------|-------------------------|
| **Rick**                | **Morty**               |
| Compute point $ n_R Q_M $ | Compute point $ n_M Q_R $ |

| **Shared Key = $ n_R n_M P $** |  |


In [ ]:
class EllipticCurveDiffieHellman:
  def __init__(self, curve, P):
    self.curve = curve
    self.P = P

  def key_generation(self, nA):
    return self.curve.point_multiply(nA, self.P)

  def key_exchange(self, nA, nB,QA ,QB):
    KA = self.curve.point_multiply(nA, QB)
    KB = self.curve.point_multiply(nB, QA)
    return KA, KB

In [ ]:
p = 3851
a = 324
b = 1287
curve = EllipticCurve(p, a, b)

P = (920, 303)              # Public point
DH = EllipticCurveDiffieHellman(curve, P)

nA = 1194                   # Morty's Private key
nB = 1759                   # Rick's Private Key
QA = DH.key_generation(nA)  # Morty's Public Key
QB = DH.key_generation(nB)  # Rick's Public Key

KA, KB = DH.key_exchange(nA, nB, QA, QB)

print("Morty's Public Key:", QA)
print("Rick's Public Key:", QB)
print("Morty's Shared Key:", KA)
print("Rick's Shared Key:", KB)

# Verify that the shared keys are equal
assert KA == KB, "Shared keys do not match!"

Morty's Public Key: (2067, 2178)
Rick's Public Key: (3684, 3125)
Morty's Shared Key: (3347, 1242)
Rick's Shared Key: (3347, 1242)


#### ECEG


**Summary of EEG:**

| **Private Parameter Setup** |  |
|-----------------------------|--|
| Choose a large prime $ p $ and a prime-order elliptic curve $ E $ over $ \mathbb{F}_p $. Select a base point $ P $ on $ E(\mathbb{F}_p) $. |  |

| **Private Computation**     | **Private Computation**     |
|-----------------------------|-----------------------------|
| **Rick**                    | **Morty**                   |

| **Key Creation**            |                             |
|-----------------------------|-----------------------------|
| Choose a private integer $ n_R $ |                         |
| Compute $ Q_R = n_R P_R $ in $ E(\mathbb{F}_p) $ |        |
| Publish $ Q_R $            |                             |

| **Encryption**              |                             |
|-----------------------------|-----------------------------|
|                             | Choose Plaintext $ M \in E( \mathbb{F}_p) $. |
|                             | Choose Private key $ k_M $  |
|                             | Use $ Q_R $ to              |
|                             | Compute $ C_1 = k_M P \in E( \mathbb{F}_p) $ |
|                             | and $ C_2 = M + k_M Q_R $   |
|                             | Send Ciphertext $(C_1, C_2)$ to Rick |

| **Decryption**              |                             |
|-----------------------------|-----------------------------|
| Compute $ C_2 - n_R C_1 \in E(\mathbb{F}_p) $ . |          |
| Morty gets $ M $ .         |                             |


In [ ]:
class EllipticCurveElGamal:
    def __init__(self, curve, P):
        self.curve = curve
        self.P = P  # base point

    def key_generation(self, nA):
        QA = self.curve.point_multiply(nA, self.P)
        return QA  # public key

    def encryption(self, QA, M, k):
        C1 = self.curve.point_multiply(k, self.P)
        C2 = self.curve.point_addition(M, self.curve.point_multiply(k, QA))
        return C1, C2

    def decryption(self, nA, C1, C2):
        S = self.curve.point_multiply(nA, C1)
        M = self.curve.point_addition(C2, (S[0], -S[1]))  # C2 - nA*C1
        return M

In [ ]:
p = 179
a = -15
b = 18
P = (124, 143)

curve = EllipticCurve(p, a, b)
elgamal = EllipticCurveElGamal(curve, P)

nA = 137                           # Morty's private key
QA = elgamal.key_generation(nA)

M = (87, 76)                       # Message

nB = 151                           # Rick's Private Key

C1, C2 = elgamal.encryption(QA, M, nB)

decrypted_message = elgamal.decryption(nA, C1, C2)

print(f"Morty's Public Key: {QA}")
print(f"Original Message: {M}")
print(f"Ciphertext: (C1: {C1}, C2: {C2})")
print(f"Decrypted Message: {decrypted_message}")

Morty's Public Key: (50, 159)
Original Message: (87, 76)
Ciphertext: (C1: (91, 31), C2: (123, 103))
Decrypted Message: (87, 76)


#### ECMV

**Summary of Menezes-Vanstone:**

| **Private Parameter Setup**                               |  |
|----------------------------------------------------------|--|
| Choose a large prime $ p $ and a prime-order elliptic curve $ E $ over $ \mathbb{F}_p $. Select a base point $ P $ on $ E(\mathbb{F}_p) $. |  |

| **Private Computation**                                   | **Private Computation**                                   |
|----------------------------------------------------------|----------------------------------------------------------|
| **Rick**                                                  | **Morty**                                                |

| **Key Generation**                                        |                                                          |
|----------------------------------------------------------|----------------------------------------------------------|
| Choose a private integer $ n_R $                         |                                                          |
| Compute $ Q_R = n_R P $ in $ E(\mathbb{F}_p) $          |                                                          |

| **Encryption**                                           |                                                          |
|----------------------------------------------------------|----------------------------------------------------------|
|                                                          | Choose Plaintext $ m \in E(\mathbb{F}_p) $.              |
|                                                          | Choose Private key $ k_M $.                              |
|                                                          | Compute $ R = k_M P $ in $ E(\mathbb{F}_p) $.           |
|                                                          | Compute $ S = k_M Q_R $ in $ E(\mathbb{F}_p) $.         |
|                                                          | Let $ (x_S, y_S) = S $.                                  |
|                                                          | Compute $ c_1 = (x_S \cdot m_1) \mod p $.               |
|                                                          | Compute $ c_2 = (y_S \cdot m_2) \mod p $.               |
|                                                          | Send Ciphertext $(R, c_1, c_2)$ to Rick.                 |

| **Decryption**                                           |                                                          |
|----------------------------------------------------------|----------------------------------------------------------|
| Compute $ T = n_R R $ in $ E(\mathbb{F}_p) $.           |                                                          |
| Let $ (x_T, y_T) = T $.                                 |                                                          |
| Compute $ m_1 = (x_T^{-1} \cdot c_1) \mod p $.         |                                                          |
| Compute $ m_2 = (y_T^{-1} \cdot c_2) \mod p $.         |                                                          |
| Morty gets $ m = (m_1, m_2) $.                          |                                                          |


In [ ]:
class EllipticCurveMenezesVanstone:
    def __init__(self, curve, P):
        self.curve = curve
        self.P = P

    def key_generation(self, nA):
        QA = self.curve.point_multiply(nA, self.P)
        return QA

    def encryption(self, QA, m, k):
        R = self.curve.point_multiply(k, self.P)
        S = self.curve.point_multiply(k, QA)
        xS, yS = S
        c1 = (xS * m[0]) % self.curve.p
        c2 = (yS * m[1]) % self.curve.p
        return R, c1, c2

    def decryption(self, nA, R, c1, c2):
        T = self.curve.point_multiply(nA, R)
        xT, yT = T
        m1 = (pow(xT, -1, self.curve.p) * c1) % self.curve.p
        m2 = (pow(yT, -1, self.curve.p) * c2) % self.curve.p
        m = (m1, m2)
        return m

In [ ]:
p = 3851
a = 324
b = 1287
P = (920, 303)

curve = EllipticCurve(p, a, b)
ECMV = EllipticCurveMenezesVanstone(curve, P)

nA = 1194                           # Morty's private key
QA = ECMV.key_generation(nA)

M = (2011, 2550)                    # Message

nB = 1759                           # Rick's Private Key

R, C1, C2 = ECMV.encryption(QA, M, nB)

decrypted_message = ECMV.decryption(nA, R, C1, C2)

print(f"Morty's Public Key: {QA}")
print(f"Original Message: {M}")
print(f"Ciphertext: (C1: {C1}, C2: {C2})")
print(f"Decrypted Message: {decrypted_message}")

Morty's Public Key: (2067, 2178)
Original Message: (2011, 2550)
Ciphertext: (C1: 3120, C2: 1578)
Decrypted Message: (2011, 2550)
